In [1]:
import requests
from time import sleep, strftime
import random
from random import randint
from bs4 import BeautifulSoup
from selenium import webdriver
import shutil
import pickle
import os
import pandas as pd

In [2]:
def wait():
    sleep(randint(2,5))
#testing
chrome_driver = r"chromedriver.exe"
navegador = webdriver.Chrome(executable_path=chrome_driver)
navegador.get("https://todocuadros.es") 
wait()

In [4]:
#script begins
def wait():
    sleep(randint(2,5))

#dataframe without columns, columns will be filled by html attributes on website
df=pd.DataFrame()
chrome_driver = r"chromedriver.exe"
navegador = webdriver.Chrome(executable_path=chrome_driver)
navegador.get("https://todocuadros.es") 
wait()
pintores=["Monet"]#,"Rembrandt", "Rafael", "Cezanne", "Van Gogh"]
ncuadros=2
crear=False

for pintor in pintores: 
    crear=True
    busq = navegador.find_element_by_id("search")
    busq.clear()
    busq.send_keys(pintor)
    wait()
    busq.submit()
    soup = BeautifulSoup(navegador.page_source)
    soup.find_all("span", class_="snize-overhidden")[0].find("span").text
    cuad=[]
    #bucle para coger nombres de los cuadros, se da permiso a crear carpeta
    for i in range(ncuadros):
        #título
        cuad.append(soup.find_all("span", class_="snize-overhidden")[i].find("span").text)
    #bucle para descargarse las imágenes de los cuadros al final se quita permiso a creardir
    for i in range(ncuadros):
        wait()
        navegador.find_element_by_partial_link_text(cuad[-i]).click()
        pic={}
        im=str(pintor)+'/'+str(pintor)+str(-i)
        pic["Path"]=im
        wait()
        soup = BeautifulSoup(navegador.page_source)
        #aquí coge las variables
        items=soup.find_all("div", class_="item-relacionado")
        print(items)
        pic={}
        for item in items:
            pic[item.text.split(":")[0]]=item.text.split(":")[1]
        img_cuadro = soup.find("div", class_="magic-slide mt-active").find("a")["href"]
        wait()
        resp = requests.get(img_cuadro, stream=True)
        if(crear):
            os.mkdir(pintor)        
        df=df.append(pic, ignore_index=True)
        #archivo/ruta
        imagenpintor = open(im+'.jpg', 'wb')
        resp.raw.decode_content = True
        shutil.copyfileobj(resp.raw, imagenpintor)
        imagenpintor.close()
        navegador.execute_script("window.history.go(-1)")
        crear=False

[<div class="item-relacionado">Autor: <a href="/pintores-famosos/monet/" id="relac-monet">Claude Monet</a></div>, <div class="item-relacionado">Estilo: <a href="/estilos-arte/impresionismo/" id="relac-impres">Impresionismo</a></div>, <div class="item-relacionado">Tema: Naturaleza</div>, <div class="item-relacionado">Título (inglés): <em>The Artist's Garden at Vétheuil</em></div>, <div class="item-relacionado">Tipo: Cuadro</div>, <div class="item-relacionado">Técnica: Óleo</div>, <div class="item-relacionado">Soporte: Lienzo</div>, <div class="item-relacionado">Año: 1880</div>, <div class="item-relacionado">Se encuentra en: Galería Nacional de Arte, Washington</div>]
[<div class="item-relacionado">Autor: <a href="/pintores-famosos/monet/" id="relac-monet">Claude Monet</a></div>, <div class="item-relacionado">Estilo: <a href="/estilos-arte/impresionismo/" id="relac-impres">Impresionismo</a></div>, <div class="item-relacionado">Título original: <em>Charing Cross Bridge</em></div>, <div cl

In [5]:
df.head(2)

,Autor,Año,Estilo,Se encuentra en,Soporte,Tema,Tipo,Técnica,Título (inglés),Título original
0,Claude Monet,1880,Impresionismo,"Galería Nacional de Arte, Washington",Lienzo,Naturaleza,Cuadro,Óleo,The Artist's Garden at Vétheuil,NaN
1,Claude Monet,1899,Impresionismo,"Museo Thyssen-Bornemisza, Madrid",Lienzo,NaN,Cuadro,Óleo,NaN,Charing Cross Bridge
